In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import datetime
import io

from PIL import Image
from IPython.display import Image as IPyImage
import imageio

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.15.0


In [7]:
def get_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(1, activation='linear', input_dim=784))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1), loss='mean_squared_error', metrics=['mae'])
  return model

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  def on_train_batch_begin(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

  def on_train_batch_end(self, batch, logs=None):
    print('Training: batch {} begins at {}'.format(batch, datetime.datetime.now().time()))

In [5]:
model = get_model()
_ = model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          steps_per_epoch=5,
          verbose=0,
          callbacks=[MyCustomCallback()])

Training: batch 0 begins at 07:03:20.049846
Training: batch 0 begins at 07:03:20.704132
Training: batch 1 begins at 07:03:20.705801
Training: batch 1 begins at 07:03:20.710639
Training: batch 2 begins at 07:03:20.711781
Training: batch 2 begins at 07:03:20.716077
Training: batch 3 begins at 07:03:20.717284
Training: batch 3 begins at 07:03:20.720462
Training: batch 4 begins at 07:03:20.721017
Training: batch 4 begins at 07:03:20.723269


### An overview of callback methods

In [6]:
callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs:
    print("Epoch: {}, Val/Train loss ratio: {:.2f}".format(epoch, logs["val_loss"] / logs["loss"]))
)
model = get_model()
_ = model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=64,
              epochs=3,
              verbose=0,
              callbacks=[callback])

Epoch: 0, Val/Train loss ratio: 0.71
Epoch: 1, Val/Train loss ratio: 0.92
Epoch: 2, Val/Train loss ratio: 0.93


In [9]:
callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs:
    print("Epoch: {}, {}, {}".format(epoch, logs["val_loss"] , logs["loss"]))
)
model = get_model()
_ = model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=64,
              epochs=3,
              verbose=1,
              )

Epoch 1/3
938/938 [==============================] - 3s 2ms/step - loss: 32.8785 - mae: 4.7456 - val_loss: 11.6505 - val_mae: 2.8370
Epoch 2/3
938/938 [==============================] - 2s 2ms/step - loss: 32.5582 - mae: 4.7710 - val_loss: 37.1891 - val_mae: 5.4380
Epoch 3/3
938/938 [==============================] - 3s 3ms/step - loss: 31.9662 - mae: 4.6211 - val_loss: 20.5820 - val_mae: 3.9568


In [12]:
callback = tf.keras.callbacks.LambdaCallback(
    on_epoch_end=lambda epoch, logs:
    print("Epoch: {}, {}".format(epoch, logs))
)
model = get_model()
_ = model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=64,
              epochs=3,
              verbose=0,
              callbacks=[callback])

Epoch: 0, {'loss': 33.07736587524414, 'mae': 4.750387191772461, 'val_loss': 31.705738067626953, 'val_mae': 5.098238468170166}
Epoch: 1, {'loss': 32.320308685302734, 'mae': 4.7102766036987305, 'val_loss': 33.98345184326172, 'val_mae': 5.211745262145996}
Epoch: 2, {'loss': 32.37199020385742, 'mae': 4.7023396492004395, 'val_loss': 9.378778457641602, 'val_mae': 2.5070841312408447}


In [ ]:
class DetectiveOverfittingCallback(tf.keras.callbacks.Callback):
  def __init__(self, threshold=0.7):
    super(DetectiveOverfittingCallback)